In [1]:
import pandas as pd
import numpy as np

from mlpckg.stat_method import *
from mlpckg.util_method import *
from mlpckg.ML_method import *

from tqdm import tqdm
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss
from sklearn.metrics import cohen_kappa_score
from matplotlib import pyplot as plt

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')


In [2]:
# select_key() argument ("1.novem/ 2.trans/ 3.bipun/ 4.paren ")
# select_key() return (target_species,key)

num = 1
target_species,key= select_key(num)[0], select_key(num)[1]

In [3]:
shap_var = pd.read_csv("./Data/3rd_manipulated_data/shap_values/"+target_species+"shap_values_top15.csv")
shap_var = list(shap_var.name.values)

In [4]:
df_target = pd.read_csv("./Data/2nd_manipulated_data/20220808_" + str(key)+"_train_minmax.csv")
df_target = df_target.drop(df_target[(df_target[target_species + str('_18')]!=0)&(df_target['Species']!= target_species)].index).reset_index(drop=True)


df_target.Source[df_target[df_target.Source == 'INAT'].index] = 'I-naturalist'
df_target.Source[df_target[df_target.Source == 'LLP'].index] = 'The Lost Ladybugs'


delivered_package = set(df_target.columns.tolist()) - set(['State','Latitude','Longitude','public_positional_accuracy', str(target_species+str('_18'))])
delivered_package = list(delivered_package)

df_target = df_target[delivered_package]

df_target = df_target.sort_values(by='Species').reset_index(drop=True)
df_target['target'] = 0
df_target.loc[df_target.Species==target_species,'target']= 1
df_target = df_target.sort_values(by='target', ascending=False).reset_index(drop=True)



df_target.rename(columns={"Species":"Species_18"},inplace=True)
df_target.columns = [i.replace(" ","_").replace("-","") for i in df_target.columns]


In [6]:
StructOrTime = 'Time'

if StructOrTime == 'Struct':
    
    df_train, df_test,train_len,test_len = structgen(df_target=df_target,Sourcelist=['I-naturalist','The Lost Ladybugs','Bugguide.net'])

elif StructOrTime =='Time':
# Specify a year and your dataset will be divided into before and after. 
    df_train, df_test,train_len,test_len = timegen(df_target=df_target,target_species=target_species,sliceYear=2019,option=0)


Year    0.70679
dtype: float64


In [7]:
hot = list(range (0, 50))
pot = list(range(0, 50))

df_score_all = pd.DataFrame({"accuracy":[],"precision":[], "recall":[], "f1":[], "auc":[], "brier":[], "kappa":[]})
df_score_block = pd.DataFrame({"accuracy":[],"precision":[], "recall":[], "f1":[], "auc":[], "brier":[], "kappa":[]})

accuracys, precisions, recalls ,f1s, roc_aucs, brier_score, kappa_score = 0,0,0,0,0,0,0

accuracy_all=[]
precision_all=[]
recall_all=[]
f1score_all=[]
auc_all=[]
kappa_all=[]
brier_all=[]

accuracy_block=[]
precision_block=[]
recall_block=[]
f1score_block=[]
auc_block=[]
kappa_block=[]
brier_block=[]


for k in tqdm(hot):
    BOX = df_train.drop(['Species_18','Source','Year'],axis=1)
    absence_pool = BOX[train_len:].sample(n=train_len, random_state = k)
    Ml_pool = pd.concat([BOX[:train_len], absence_pool])
    answer = Ml_pool.target
    Ml_pool = Ml_pool[shap_var]


    TESTBOX = df_test.drop(['Species_18','Source','Year'],axis=1)
    test_absence = TESTBOX[test_len:].sample(n=test_len, random_state = k)
    test_pool = pd.concat([TESTBOX[:test_len], test_absence])
    test_answer = test_pool.target
    test_pool = test_pool[shap_var]


    for i in pot:
        
        X_train, _, y_train, _ = train_test_split(Ml_pool, answer, test_size=0.00000000001, random_state= i)
        _, X_test, _, y_test= train_test_split(test_pool, test_answer, test_size=0.96, random_state= i)
        
       
       
        xgb_wrapper = XGBClassifier(n_estimators=1000, learning_rate=0.7, max_depth=7, objective = "binary:logistic")
        evals = [(X_test, y_test)]
        xgb_wrapper.fit(X_train, y_train, early_stopping_rounds=10, 
                            eval_metric="error", eval_set=evals, verbose=0)

                            
        ws100_preds = xgb_wrapper.predict(X_test)
        ws100_pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

        a, b, c, d, e = get_clf_eval_num(y_test, ws100_preds, ws100_pred_proba)
        accuracys = accuracys + a
        precisions = precisions + b
        recalls = recalls + c
        f1s = f1s + d
        roc_aucs = roc_aucs + e
        
        f = brier_score_loss(y_test, ws100_pred_proba)
        g = cohen_kappa_score(ws100_preds, y_test)
        brier_score = brier_score + f
        kappa_score = kappa_score + g
        
        accuracy_all.append(a)
        precision_all.append(b)
        recall_all.append(c)
        f1score_all.append(d)
        auc_all.append(e)
        brier_all.append(f)
        kappa_all.append(g)
        

    accuracy_block.append(accuracys/len(pot))
    precision_block.append(precisions/len(pot))
    recall_block.append(recalls/len(pot))
    f1score_block.append(f1s/len(pot))
    auc_block.append(roc_aucs/len(pot))
    kappa_block.append(kappa_score/len(pot))
    brier_block.append(brier_score/len(pot))

    accuracys = 0
    precisions = 0
    recalls = 0
    f1s = 0
    roc_aucs = 0
    brier_score = 0
    kappa_score = 0



df_score_all['accuracy']=accuracy_all
df_score_all['precision']=precision_all
df_score_all['recall']=recall_all
df_score_all['f1']=f1score_all
df_score_all['auc']=auc_all
df_score_all['brier']=brier_all
df_score_all['kappa']=kappa_all

df_score_block['accuracy']=accuracy_block
df_score_block['precision']=precision_block
df_score_block['recall']=recall_block
df_score_block['f1']=f1score_block
df_score_block['auc']=auc_block
df_score_block['brier']=brier_block
df_score_block['kappa']=kappa_block


df_score_block.describe()

100%|██████████| 50/50 [01:13<00:00,  1.48s/it]


,accuracy,precision,recall,f1,auc,brier,kappa
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.803906,0.826270,0.774274,0.797698,0.864682,0.155344,0.607809
std,0.033442,0.042382,0.054270,0.036928,0.029032,0.020235,0.066873
min,0.726339,0.725023,0.625933,0.709980,0.788340,0.119114,0.452620
25%,0.783770,0.802476,0.747253,0.783932,0.844690,0.138011,0.567409
50%,0.810820,0.823596,0.786432,0.805593,0.866239,0.158098,0.621601
75%,0.827295,0.859405,0.803943,0.819146,0.884199,0.163875,0.654607
max,0.880437,0.907687,0.872877,0.878029,0.919960,0.202220,0.760849
